In [ ]:
# first upload the data.zip
!unzip data.zip &>/dev/null

In [ ]:
# download champion json from riot game website
!wget http://ddragon.leagueoflegends.com/cdn/11.1.1/data/en_US/champion.json -O champion.json

# load champion.json that just got downloaded
import json

id_to_champion = {'0':'None'}
with open('champion.json', "r", encoding='utf-8') as f:
    js = json.load(f)
    for champ_name in js['data']:
        id_to_champion[js['data'][champ_name]['key']] = champ_name
    f.close()

with open('key-to-champ.json', 'w', encoding='utf-8') as f:
    json.dump(id_to_champion, f, ensure_ascii=False, indent=4)
    f.close()

for k,v in id_to_champion.items():
    print('{}: "{}", '.format(k,v))

--2021-02-16 03:54:42--  http://ddragon.leagueoflegends.com/cdn/11.1.1/data/en_US/champion.json
Resolving ddragon.leagueoflegends.com (ddragon.leagueoflegends.com)... 52.85.147.67
Connecting to ddragon.leagueoflegends.com (ddragon.leagueoflegends.com)|52.85.147.67|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140718 (137K) [application/json]
Saving to: ‘champion.json’

champion.json       100%[===================>] 137.42K  --.-KB/s    in 0.001s  

2021-02-16 03:54:42 (99.8 MB/s) - ‘champion.json’ saved [140718/140718]

0: "None", 
266: "Aatrox", 
103: "Ahri", 
84: "Akali", 
12: "Alistar", 
32: "Amumu", 
34: "Anivia", 
1: "Annie", 
523: "Aphelios", 
22: "Ashe", 
136: "AurelionSol", 
268: "Azir", 
432: "Bard", 
53: "Blitzcrank", 
63: "Brand", 
201: "Braum", 
51: "Caitlyn", 
164: "Camille", 
69: "Cassiopeia", 
31: "Chogath", 
42: "Corki", 
122: "Darius", 
131: "Diana", 
119: "Draven", 
36: "DrMundo", 
245: "Ekko", 
60: "Elise", 
28: "Evelynn", 
81: "Ezreal", 
9

In [ ]:
# sort file by elo
import os
import shutil

for f in os.listdir("data"):
    folder_name = f.split("-")[-1][:-4]
    if not os.path.exists(folder_name):
        os.mkdir(folder_name)
        shutil.copy(os.path.join('data', f), folder_name)
    else:
        shutil.copy(os.path.join('data', f), folder_name)

Our labels are flipped in the match data pipeline. We will not correct this mistake in the pipeline as we will need to redo all the csv.


Currently the label looks like: 
```
GAME_ID	RedBan1	RedBan2	RedBan3	RedBan4	RedBan5	RedTop	RedJg	RedMid	RedAdc	RedSup	BlueBan1	BlueBan2	BlueBan3	BlueBan4	BlueBan5	BlueTop	BlueJg	BlueMid	BlueAdc	BlueSup	redTeamWin
```

However, it should read like:
```
GAME_ID	RedTop	RedJg	RedMid	RedAdc	RedSup	RedBan1	RedBan2	RedBan3	RedBan4	RedBan5	BlueTop	BlueJg	BlueMid	BlueAdc	BlueSup	BlueBan1	BlueBan2	BlueBan3	BlueBan4	BlueBan5	redTeamWin
```

The ban and pick labels are flipped for both teams.


In [ ]:
# combine the files by elo and put them to combined/
import pandas as pd

elos = [
    'IRON',
    'BRONZE',
    'SILVER',
    'GOLD',
    'PLATINUM',
    'DIAMOND',
]

rename_dict = {
    "RedBan1": "RedTop",
    "RedBan2": "RedJg",
    "RedBan3": "RedMid",
    "RedBan4": "RedAdc",
    "RedBan5": "RedSup",
    'RedAdc': 'RedBan4',
    'RedJg': 'RedBan2',
    'RedMid': 'RedBan3',
    'RedSup': 'RedBan5',
    'RedTop': 'RedBan1',
    "BlueBan1": "BlueTop",
    "BlueBan2": "BlueJg",
    "BlueBan3": "BlueMid",
    "BlueBan4": "BlueAdc",
    "BlueBan5": "BlueSup",
    "BlueAdc": "BlueBan4",
    "BlueJg": "BlueBan2",
    "BlueMid": "BlueBan3",
    "BlueSup": "BlueBan5",
    "BlueTop": "BlueBan1"
}

combined_path = "combined/"

if not os.path.exists(combined_path):
    os.mkdir(combined_path)

if not os.path.exists('crunched_data'):
    os.mkdir('crunched_data')

for elo in elos:
    files = [pd.read_csv(elo+"/"+i) for i in os.listdir(elo)]
    df = pd.concat(files)
    df.rename(columns=rename_dict, inplace=True)
    df.to_csv(combined_path+elo+".csv", index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=939b35d5-0867-4fd3-83c2-58e7fdcd481f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>